#### 1.Load pakages

In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
from tqdm import tqdm

#### 2.Load lincs data

In [2]:
import scanpy as sc

lincs_adata = sc.read('Lincs_L1000.h5ad')

#### 3.Load CCLE data

In [3]:
Expression = pd.read_csv('./SCLC/CCLE.csv', index_col=0)
col_list = Expression.columns.to_list()
col_list1 = [i.split(' (')[0] for i in col_list]
Expression.columns = col_list1


#### 4.Process the expression data and use 978 landmark gene for test

In [4]:

gene_list = lincs_adata.var_names.to_list()

adata_var = {}
for gene in gene_list:
    adata_var[gene] = gene in col_list1

adata_var_df = pd.DataFrame.from_dict(adata_var, orient='index')
adata_var_df = adata_var_df.rename(columns={0:'InTrainSet'})
other_gene = list(adata_var_df[adata_var_df.InTrainSet==False].index)
print('OOD GENE : ', len(other_gene))

for idx in Expression.index.to_list():
    for gene in other_gene:
        Expression.loc[idx, gene] = Expression.loc[idx,:].mean()

adata_x = pd.DataFrame(columns=lincs_adata.var.index.to_list(),index=Expression.index.to_list())
for idx in adata_x.index.to_list():
    for gene in adata_x.columns.to_list():
        adata_x.loc[idx,gene] = Expression.loc[idx,gene]

    

OOD GENE :  29


In [8]:
adata_X_control=adata_x.loc[['ACH-000800','ACH-000530']]
adata_X_control

,DDR1,PAX8,RPS5,ABCF1,SPAG7,RHOA,RNPS1,SMNDC1,ATP6V0B,RPS6,...,P4HTM,SLC27A3,TBXA2R,RTN2,TSTA3,PPARD,GNA11,WDTC1,PLSCR3,NPEPL1
ACH-000800,5.536053,1.321928,11.050461,6.510487,6.547357,7.898813,7.842162,4.519164,7.482768,10.783162,...,3.408712,2.89724,0.389567,5.383359,2.815564,4.134221,5.283551,4.612942,4.063503,1.130931
ACH-000530,4.185867,0.097611,10.935165,6.835293,5.635464,9.065685,7.597233,5.385086,8.102133,9.755188,...,3.963474,4.020591,1.914565,3.705978,2.982383,4.698218,5.108524,3.937344,5.246028,3.06695


In [9]:
control_obs_df = pd.DataFrame(index=adata_X_control.index)
for i in adata_X_control.index.to_list():
    cell_line = i
    control_obs_df.loc[cell_line,'Drug'] = 'DMSO'
    control_obs_df.loc[cell_line,'dose'] = 0.0
    control_obs_df.loc[cell_line,'SMILES'] = 'unknown'
    control_obs_df.loc[cell_line,'cell_type'] = cell_line
    control_obs_df.loc[cell_line,'dose'] = 0.0
    control_obs_df.loc[cell_line,'cov_drug_name'] = cell_line+'_DMSO'
    control_obs_df.loc[cell_line,'cov_drug_dose_name'] = cell_line+'_DMSO_0.0'

In [10]:
control_obs_df

,Drug,dose,SMILES,cell_type,cov_drug_name,cov_drug_dose_name
ACH-000800,DMSO,0.0,unknown,ACH-000800,ACH-000800_DMSO,ACH-000800_DMSO_0.0
ACH-000530,DMSO,0.0,unknown,ACH-000530,ACH-000530_DMSO,ACH-000530_DMSO_0.0


In [11]:
import anndata as ad
adata_control = ad.AnnData(adata_X_control,obs=control_obs_df,var=lincs_adata.var)
adata_control

/home/nicole/anaconda3/envs/GEARS/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  


AnnData object with n_obs × n_vars = 2 × 978
    obs: 'Drug', 'dose', 'SMILES', 'cell_type', 'cov_drug_name', 'cov_drug_dose_name'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'

#### 5.Process the compound data 

In [13]:
# You can change the compound library 
FDA_drug_df = pd.read_csv('./FDA_961/Brief_FDA-Approved-Drug_961.csv', index_col=0)

In [14]:
adata_obs_df = pd.DataFrame()
for cell_type in control_obs_df.index.to_list():
    print(cell_type)
    
    dose_list = [1, 10]
    adata_obs_df_ = pd.DataFrame()

    FDA_drug_df['cell_type'] = cell_type

    for i in range(len(dose_list)):
        FDA_drug_df['dose'] = dose_list[i]
        adata_obs_df_ = pd.concat([adata_obs_df_, FDA_drug_df], ignore_index=True)
    
    adata_obs_df = pd.concat([adata_obs_df, adata_obs_df_], ignore_index=True)


adata_obs_df

ACH-000800
ACH-000530


,ID,Name,CAS Number,SMILES,cell_type,dose
0,S1001,Navitoclax (ABT-263),923564-51-6,CC1(C)CCC(=C(CN2CCN(CC2)C2=CC=C(C=C2)C(=O)NS(=...,ACH-000800,1
1,S1003,Linifanib (ABT-869),796967-16-3,CC1=CC(NC(=O)NC2=CC=C(C=C2)C2=C3C(N)=NNC3=CC=C...,ACH-000800,1
2,S1004,Veliparib (ABT-888),912444-00-9,C[C@@]1(CCCN1)C1=NC2=C(C=CC=C2N1)C(N)=O,ACH-000800,1
3,S1005,Axitinib,319460-85-0,CNC(=O)C1=CC=CC=C1SC1=CC2=C(C=C1)C(\C=C\C1=CC=...,ACH-000800,1
4,S1006,Saracatinib (AZD0530),379231-04-6,CN1CCN(CCOC2=CC(OC3CCOCC3)=C3C(NC4=C(Cl)C=CC5=...,ACH-000800,1
...,...,...,...,...,...,...
3839,S4653,L-Ornithine,70-26-8,NCCC[C@H](N)C(O)=O,ACH-000530,10
3840,S4671,Cefradine,38821-53-3,[H][C@]12SCC(C)=C(N1C(=O)[C@H]2NC(=O)[C@H](N)C...,ACH-000530,10
3841,S4684,Sildenafil,139755-83-2,CCCC1=NN(C)C2=C1N=C(NC2=O)C1=CC(=CC=C1OCC)S(=O...,ACH-000530,10
3842,S4700,4-Aminobutyric acid,56-12-2,NCCCC(O)=O,ACH-000530,10


In [15]:
adata_obs_df['Drug'] = adata_obs_df['ID']
adata_obs_df['cov_drug_name'] = adata_obs_df['cell_type'].astype(str)+ '_' + adata_obs_df['Drug'].astype(str)
adata_obs_df['cov_drug_dose_name'] = adata_obs_df['cell_type'].astype(str)+ '_' + adata_obs_df['Drug'].astype(str)+ '_' + adata_obs_df['dose'].astype(str)
adata_obs_df

,ID,Name,CAS Number,SMILES,cell_type,dose,Drug,cov_drug_name,cov_drug_dose_name
0,S1001,Navitoclax (ABT-263),923564-51-6,CC1(C)CCC(=C(CN2CCN(CC2)C2=CC=C(C=C2)C(=O)NS(=...,ACH-000800,1,S1001,ACH-000800_S1001,ACH-000800_S1001_1
1,S1003,Linifanib (ABT-869),796967-16-3,CC1=CC(NC(=O)NC2=CC=C(C=C2)C2=C3C(N)=NNC3=CC=C...,ACH-000800,1,S1003,ACH-000800_S1003,ACH-000800_S1003_1
2,S1004,Veliparib (ABT-888),912444-00-9,C[C@@]1(CCCN1)C1=NC2=C(C=CC=C2N1)C(N)=O,ACH-000800,1,S1004,ACH-000800_S1004,ACH-000800_S1004_1
3,S1005,Axitinib,319460-85-0,CNC(=O)C1=CC=CC=C1SC1=CC2=C(C=C1)C(\C=C\C1=CC=...,ACH-000800,1,S1005,ACH-000800_S1005,ACH-000800_S1005_1
4,S1006,Saracatinib (AZD0530),379231-04-6,CN1CCN(CCOC2=CC(OC3CCOCC3)=C3C(NC4=C(Cl)C=CC5=...,ACH-000800,1,S1006,ACH-000800_S1006,ACH-000800_S1006_1
...,...,...,...,...,...,...,...,...,...
3839,S4653,L-Ornithine,70-26-8,NCCC[C@H](N)C(O)=O,ACH-000530,10,S4653,ACH-000530_S4653,ACH-000530_S4653_10
3840,S4671,Cefradine,38821-53-3,[H][C@]12SCC(C)=C(N1C(=O)[C@H]2NC(=O)[C@H](N)C...,ACH-000530,10,S4671,ACH-000530_S4671,ACH-000530_S4671_10
3841,S4684,Sildenafil,139755-83-2,CCCC1=NN(C)C2=C1N=C(NC2=O)C1=CC(=CC=C1OCC)S(=O...,ACH-000530,10,S4684,ACH-000530_S4684,ACH-000530_S4684_10
3842,S4700,4-Aminobutyric acid,56-12-2,NCCCC(O)=O,ACH-000530,10,S4700,ACH-000530_S4700,ACH-000530_S4700_10


In [19]:
adata_drug_X = np.zeros((adata_obs_df.shape[0], 978))

adata_test = ad.AnnData(adata_drug_X,obs=adata_obs_df,var=lincs_adata.var)
adata_test

/home/nicole/anaconda3/envs/GEARS/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  This is separate from the ipykernel package so we can avoid doing imports until


AnnData object with n_obs × n_vars = 3844 × 978
    obs: 'ID', 'Name', 'CAS Number', 'SMILES', 'cell_type', 'dose', 'Drug', 'cov_drug_name', 'cov_drug_dose_name'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'

#### 6.Concat the control and test data 

In [20]:
adata_obs_all = pd.concat([adata_control.obs, adata_test.obs], ignore_index=True)
adata_obs_all.index = adata_obs_all.index.astype(str)
adata_X_all = np.concatenate((adata_control.X, adata_test.X),axis=0)

adata = ad.AnnData(adata_X_all,obs=adata_obs_all,var=lincs_adata.var)
adata

AnnData object with n_obs × n_vars = 3846 × 978
    obs: 'Drug', 'dose', 'SMILES', 'cell_type', 'cov_drug_name', 'cov_drug_dose_name', 'ID', 'Name', 'CAS Number'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'

In [21]:
adata.obs.loc[adata.obs.Drug!='DMSO', 'CCLE_split']='test'
adata.obs.loc[adata.obs.Drug=='DMSO', 'CCLE_split']=''
adata.obs

,Drug,dose,SMILES,cell_type,cov_drug_name,cov_drug_dose_name,ID,Name,CAS Number,CCLE_split
0,DMSO,0.0,unknown,ACH-000800,ACH-000800_DMSO,ACH-000800_DMSO_0.0,NaN,NaN,NaN,
1,DMSO,0.0,unknown,ACH-000530,ACH-000530_DMSO,ACH-000530_DMSO_0.0,NaN,NaN,NaN,
2,S1001,1.0,CC1(C)CCC(=C(CN2CCN(CC2)C2=CC=C(C=C2)C(=O)NS(=...,ACH-000800,ACH-000800_S1001,ACH-000800_S1001_1,S1001,Navitoclax (ABT-263),923564-51-6,test
3,S1003,1.0,CC1=CC(NC(=O)NC2=CC=C(C=C2)C2=C3C(N)=NNC3=CC=C...,ACH-000800,ACH-000800_S1003,ACH-000800_S1003_1,S1003,Linifanib (ABT-869),796967-16-3,test
4,S1004,1.0,C[C@@]1(CCCN1)C1=NC2=C(C=CC=C2N1)C(N)=O,ACH-000800,ACH-000800_S1004,ACH-000800_S1004_1,S1004,Veliparib (ABT-888),912444-00-9,test
...,...,...,...,...,...,...,...,...,...,...
3841,S4653,10.0,NCCC[C@H](N)C(O)=O,ACH-000530,ACH-000530_S4653,ACH-000530_S4653_10,S4653,L-Ornithine,70-26-8,test
3842,S4671,10.0,[H][C@]12SCC(C)=C(N1C(=O)[C@H]2NC(=O)[C@H](N)C...,ACH-000530,ACH-000530_S4671,ACH-000530_S4671_10,S4671,Cefradine,38821-53-3,test
3843,S4684,10.0,CCCC1=NN(C)C2=C1N=C(NC2=O)C1=CC(=CC=C1OCC)S(=O...,ACH-000530,ACH-000530_S4684,ACH-000530_S4684_10,S4684,Sildenafil,139755-83-2,test
3844,S4700,10.0,NCCCC(O)=O,ACH-000530,ACH-000530_S4700,ACH-000530_S4700_10,S4700,4-Aminobutyric acid,56-12-2,test


In [22]:
for cell_type in adata.obs.cell_type.unique().tolist():
    adata.obs.loc[(adata.obs.Drug!='DMSO') & (adata.obs.cell_type==cell_type), 'paired_control_index']=adata.obs[(adata.obs.Drug=='DMSO') & (adata.obs.cell_type==cell_type)].index.values[0]
adata.obs

,Drug,dose,SMILES,cell_type,cov_drug_name,cov_drug_dose_name,ID,Name,CAS Number,CCLE_split,paired_control_index
0,DMSO,0.0,unknown,ACH-000800,ACH-000800_DMSO,ACH-000800_DMSO_0.0,NaN,NaN,NaN,,NaN
1,DMSO,0.0,unknown,ACH-000530,ACH-000530_DMSO,ACH-000530_DMSO_0.0,NaN,NaN,NaN,,NaN
2,S1001,1.0,CC1(C)CCC(=C(CN2CCN(CC2)C2=CC=C(C=C2)C(=O)NS(=...,ACH-000800,ACH-000800_S1001,ACH-000800_S1001_1,S1001,Navitoclax (ABT-263),923564-51-6,test,0
3,S1003,1.0,CC1=CC(NC(=O)NC2=CC=C(C=C2)C2=C3C(N)=NNC3=CC=C...,ACH-000800,ACH-000800_S1003,ACH-000800_S1003_1,S1003,Linifanib (ABT-869),796967-16-3,test,0
4,S1004,1.0,C[C@@]1(CCCN1)C1=NC2=C(C=CC=C2N1)C(N)=O,ACH-000800,ACH-000800_S1004,ACH-000800_S1004_1,S1004,Veliparib (ABT-888),912444-00-9,test,0
...,...,...,...,...,...,...,...,...,...,...,...
3841,S4653,10.0,NCCC[C@H](N)C(O)=O,ACH-000530,ACH-000530_S4653,ACH-000530_S4653_10,S4653,L-Ornithine,70-26-8,test,1
3842,S4671,10.0,[H][C@]12SCC(C)=C(N1C(=O)[C@H]2NC(=O)[C@H](N)C...,ACH-000530,ACH-000530_S4671,ACH-000530_S4671_10,S4671,Cefradine,38821-53-3,test,1
3843,S4684,10.0,CCCC1=NN(C)C2=C1N=C(NC2=O)C1=CC(=CC=C1OCC)S(=O...,ACH-000530,ACH-000530_S4684,ACH-000530_S4684_10,S4684,Sildenafil,139755-83-2,test,1
3844,S4700,10.0,NCCCC(O)=O,ACH-000530,ACH-000530_S4700,ACH-000530_S4700_10,S4700,4-Aminobutyric acid,56-12-2,test,1


In [ ]:
#### 7.Save custom dataset 

In [23]:
adata.obs.index = adata.obs.index.astype(str)
adata.write('../dataset/CCLE_split.h5ad')